In [ ]:
import pandas as pd
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, BertModel
import torch
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from collections import Counter
import re
import numpy as np
import spacy
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import stats
from wordcloud import WordCloud

In [ ]:
# Загрузка необходимых ресурсов NLTK и spacy
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nlp = spacy.load("ru_core_news_md")

In [3]:
"""
MAPPING
"""

# Создание словаря для сопоставления кодов стран с полными названиями
country_mapping = {
    'RU': 'Россия',
    'BY': 'Беларусь',
    'KZ': 'Казахстан',
    'UZ': 'Узбекистан',
    'KG': 'Киргизия'
}

# Создание словаря для сопоставления типов занятости с полными названиями
employment_type_mapping = {
    'FULL_TIME': 'Полная занятость',
    'PART_TIME': 'Частичная занятость',
    'TEMPORARY': 'Проектная работа'
}

categories = {
    'Менеджер': ['менеджер', 'управляющий', 'руководитель', 'manager', 'директор', 'начальник'],
    'Водитель': ['водитель', 'экспедитор', 'курьер', 'шофер'],
    'Продавец': ['продавец', 'кассир', 'мерчендайзер', 'консультант', 'заказ', 'заказы'],
    'Инженер': ['инженер', 'техник', 'механик', 'монтажник'],
    'Питание': ['повар', 'сушист', 'бариста', 'кондитер', 'официант', 'бармен', 'сушеф'],
    'Офисный персонал': ['офис-менеджер', 'ассистент', 'секретарь', 'администратор', 'бухгалтер', 'экономист', 'оператор', 'call', 'хостес', 'юрист', 'кадры', 'hr', 'снабженец', 'логист', 'переводчик', 'маркетолог', 'продажи', 'продажа', 'рекрутер', 'редактор', 'кадр'],
    'IT специалист': ['разработчик', 'программист', 'системный администратор', 'аналитик', 'дизайнер', 'тестировщик'],
    'Образование': ['учитель', 'преподаватель', 'тренер', 'педагог', 'воспитатель', 'няня'],
    'Работа на складе': ['сборщик', 'кладовщик', 'упаковщик', 'склад', 'комплектовщик', 'грузчик'],
    'Уборка': ['уборка', 'уборщик', 'уборщица', 'дворник', 'клининг', 'горничная', 'дворецкий', 'мойщик'],
    'Медицина': ['хирург', 'стоматолог', 'врач'],
    'Охрана': ['охранник', 'охрана', 'безопасность'],
}

In [4]:
def clean_description_to_plain_text(text):
    """
    Функция для очистки текста: удаляет HTML-теги, маркеры списков, обрабатывает переносы строк,
    устраняет лишние символы и пробелы, а также подготавливает текст для сохранения.
    """
    # Убираем HTML-теги
    text = re.sub(r"<[^>]+>", " ", text)

    # Заменяем спецсимволы (\n, \\n) на пробелы
    text = text.replace("\\n", " ").replace("\n", " ")

    # Убираем маркеры "•" и заменяем их на пробел
    text = re.sub(r"•", " ", text)

    # Заменяем нулевые пробелы (\u200b) на обычные пробелы
    text = text.replace("\u200b", " ")

    # Убираем одиночные точки, не связанные с текстом
    text = re.sub(r"(?<!\w)\.(?!\w)", "", text)  # Убираем точки без текста вокруг
    text = re.sub(r"\s*\.\s*", ". ", text)  # Исправляем повторяющиеся точки

    # Убираем точки в конце строки
    text = re.sub(r"\s*\.$", "", text)

    # Заменяем &quot; на пробел
    text = re.sub(r"&quot;", " ", text)

    # Убираем лишние пробелы
    text = re.sub(r"\s+", " ", text)  # Сжимаем пробелы
    text = text.strip()  # Убираем пробелы в начале и конце текста

    # Экранируем кавычки для CSV
    text = text.replace('"', '""')  

    return text

In [5]:
def extract_salary_and_currency(salary_str):
    """
    Функция для выбора валюты, с помощью регулярок вычленяет валюту.
    Также учитывает НДФЛ при подсчете зп на руки.
    """
    salary_str = salary_str.replace(' ', '')
    numbers = re.findall(r'\d+', salary_str)
    currency = re.findall(r'[A-Za-zА-Яа-я₽$€₸]+', salary_str)
    currency = [c for c in currency if c not in ['от', 'до', 'довычетаналогов', 'наруки']]
    currency = currency[0] if currency else None

    if len(numbers) == 1:
        return int(numbers[0]), currency
    elif len(numbers) == 2:
        average_salary = (int(numbers[0]) + int(numbers[1])) / 2
        return average_salary, currency
    else:
        return None, currency

def clean_currency(currency_str):
    if currency_str:
        currency_str = re.sub(r'довычетаналогов|наруки', '', currency_str)
    return currency_str.strip()

def convert_salary_to_numeric(salary_str):
    if salary_str == 'Зарплата не указана':
        return 'Зарплата не указана', 'Валюта не указана'
    
    salary, currency = extract_salary_and_currency(salary_str)
    
    if salary is not None:
        if 'до вычета налогов' in salary_str.lower():
            salary *= 0.87  # Вычитаем 13%
        return salary, clean_currency(currency)
    
    return 'Зарплата не указана', clean_currency(currency)

In [ ]:
# Функция для преобразования значений
def convert_to_integer(value):
    """
    Функция преобразуют ЗП в целое число с исключением зп не найдена
    """
    if value == 'Зарплата не указана':
        return value
    else:
        try:
            # Преобразование в число и округление до целого
            return round(float(value))
        except ValueError:
            return value

In [6]:
# Курсы обмена на 21.12.2024


# Функция для перевода зарплаты в рубли
def convert_to_rub(row):
    """
    Функция конвертируют валюту в датасете в Рубли.
    """
    exchange_rates = {
    'so': 0.0079,
    'сом': 1.18,
    '$': 102.34,
    'Br': 29.69,
    '€': 109.31,
    '₸': 0.20
}
    if row['Валюта'] == '₽':
        return row['Зарплата на руки (числовое выражение)']
    elif row['Валюта'] == 'Валюта не указана':
        return 'Зарплата не указана'
    else:
        return row['Зарплата на руки (числовое выражение)'] * exchange_rates[row['Валюта']]

In [8]:
# Функция для определения категорий дискриминации с учетом контекста
def determine_discrimination_categories(text):
    """
    Функция опредлеляет вид дискриминация, учитывая контекст. Тут можно поигарть со словарем и с моделькой. Щас использую Spacy
    """
    # Словари для категорий дискриминации
    discrimination_categories = {
    'Возрастная дискриминация': ['молодость', 'старость', 'пенсионный', 'энергичность', 'активность', 'обучаемый', 'пенсия', 'возраст'],
    'Гендерная дискриминация': ['мужчина', 'женщина', 'девушка', 'парень'],
    'Этническая дискриминация': ['прописка', 'регистрация', 'внешность', 'вид', 'акцент'],
    'Иные виды дискриминации': ['предпочтительность', 'профессионал', 'командный', 'семья']
}
    doc = nlp(text)
    found_categories = set()  # Используем множество для хранения найденных категорий
    for category, words in discrimination_categories.items():
        for word in words:
            if word in text:
                # Проверяем контекст слова
                for token in doc:
                    if token.text == word:
                        found_categories.add(category)  # Добавляем категорию в множество
                        break
    return ', '.join(found_categories) if found_categories else 'Не найдено дискриминации'

In [ ]:
# Функция предобработки текста
def preprocess_text(text):
    # Удаление знаков препинания и других символов
    text = re.sub(r'[^\w\s]', '', text)
    # Удаление цифр
    text = re.sub(r'\d+', '', text)
    # Приведение текста к нижнему регистру
    text = text.lower()
    # Токенизация текста
    tokens = word_tokenize(text)
    # Загрузка стоп-слов
    stop_words = set(stopwords.words('russian'))
    # Удаление стоп-слов
    filtered_tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    # Лематизация
    doc = nlp(' '.join(filtered_tokens))
    lemmatized_tokens = [token.lemma_ for token in doc]
    return ' '.join(lemmatized_tokens)


In [ ]:
# Загрузка предобученной модели
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Создаём эмбеддинги для категорий (на основе примеров)
category_embeddings = {category: model.encode(examples) for category, examples in categories.items()}

# Функция для векторизации текста
def get_text_embedding(text):
    return model.encode([text])[0]

def categorize_job_vectorized(text):
    # Преобразуем текст в эмбеддинг
    text_embedding = get_text_embedding(text).reshape(1, -1)  # Преобразуем в 2D массив

    max_similarity = 0
    best_category = None  # Дефолтное значение, если ничего не найдено
    
    for category, embeddings in category_embeddings.items():
        # Преобразуем эмбеддинги категорий в 2D массивы
        category_embeddings_2d = [emb.reshape(1, -1) for emb in embeddings]
        
        # Считаем максимальную схожесть для текущей категории
        similarity = max([cosine_similarity(text_embedding, emb)[0][0] for emb in category_embeddings_2d])
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_category = category

    return best_category or "Неопределено"  # Если категория не найдена, возвращаем "Неопределено"


In [ ]:
# Функция для преобразования строки в строку с разделителями-запятыми
def words_to_comma_separated(text):
    return ', '.join(text.split())